# Orchestrator (ver. 0.8)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2022

***

# (GPU CHECK)

In [ ]:
#@title GPU check
!nvidia-smi

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install dependencies
!git clone https://github.com/asigalov61/Orchestrator
!pip install torch
!pip install einops
!pip install torch-summary
!pip install sklearn
!pip install tqdm
!pip install matplotlib

In [ ]:
#@title Import modules

print('Loading modules...')

import os
import pickle
import random
import secrets
import statistics
from time import time
import tqdm

import torch

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

%cd /content/Orchestrator

import TMIDIX

from lwa_transformer import *

%cd /content/

print('Done')

# (LOAD MODEL)

In [ ]:
#@title Unzip Pre-Trained Orchestrator Model
%cd /content/Orchestrator/Model

print('=' * 70)
print('Unzipping pre-trained GIGA-Piano XL model...Please wait...')

!cat /content/Orchestrator/Model/Orchestrator_Trained_Model_55253_steps_0.3277_loss.zip* > /content/Orchestrator/Model/Orchestrator_Trained_Model_55253_steps_0.3277_loss.zip
print('=' * 70)

!unzip -j /content/Orchestrator/Model/Orchestrator_Trained_Model_55253_steps_0.3277_loss.zip
print('=' * 70)

print('Done! Enjoy! :)')
print('=' * 70)
%cd /content/

In [ ]:
#@title Load Orchestrator Pre-Trained Model

print('Loading model...Please wait...')

SEQ_LEN = 4096

# instantiate the model

model = LocalTransformer(
    num_tokens = 774,
    dim = 1024,
    depth = 24,
    causal = True,
    local_attn_window_size = 512,
    max_seq_len = SEQ_LEN
).cuda()

state_dict = torch.load('/content/Orchestrator/Model/Orchestrator_Trained_Model_55253_steps_0.3277_loss.pth')

model.load_state_dict(state_dict)

model.eval()

# Model stats

print('Done!')
      
summary(model)

# Plot Token Embeddings
tok_emb = model.token_emb.weight.detach().cpu().tolist()

tok_emb1 = []

for t in tok_emb:
    tok_emb1.append([abs(statistics.median(t))])

cos_sim = metrics.pairwise_distances(
   tok_emb1, metric='euclidean'
)
plt.figure(figsize=(10, 10))
plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
plt.xlabel("Position")
plt.ylabel("Position")
plt.tight_layout()
plt.plot()
plt.savefig("/content/Orchestrator-Tokens-Embeddings-Plot.png", bbox_inches="tight")

# (LOAD SEED MIDI)

In [ ]:
#@title Load Seed MIDI
full_path_to_seed_MIDI = "/content/Orchestrator/Seeds/Orchestrator-Piano-Seed-1.mid" #@param {type:"string"}
f = full_path_to_seed_MIDI

transpose_to_model_average_pitch = False

#=======================================================
# START PROCESSING

# Convering MIDI to ms score with MIDI.py module
score = TMIDIX.midi2ms_score(open(f, 'rb').read())

# INSTRUMENTS CONVERSION CYCLE
events_matrix = []
melody_chords_f = []
melody_chords_f1 = []
itrack = 1
patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

patch_map = [[0, 1, 2, 3, 4, 5, 6, 7], # Piano 
          [24, 25, 26, 27, 28, 29, 30], # Guitar
          [32, 33, 34, 35, 36, 37, 38, 39], # Bass
          [40, 41], # Violin
          [42, 43], # Cello
          [46], # Harp
          [56, 57, 58, 59, 60], # Trumpet
          [71, 72], # Clarinet
          [73, 74, 75], # Flute
          [-1], # Drums
          [52, 53], # Choir
          [16, 17, 18, 19, 20] # Organ
          ]

while itrack < len(score):
  for event in score[itrack]:         
      if event[0] == 'note' or event[0] == 'patch_change':
          events_matrix.append(event)
  itrack += 1

events_matrix.sort(key=lambda x: x[1])

events_matrix1 = []

for event in events_matrix:
  if event[0] == 'patch_change':
      patches[event[2]] = event[3]

  if event[0] == 'note':
      event.extend([patches[event[3]]])
      once = False

      for p in patch_map:
          if event[6] in p and event[3] != 9: # Except the drums
              event[3] = patch_map.index(p)
              once = True

      if not once and event[3] != 9: # Except the drums
          event[3] = 15 # All other instruments/patches channel
          event[5] = max(80, event[5])

      if event[3] < 12: # We won't write chans 12-16 for now...
          events_matrix1.append(event)
          # stats[event[3]] += 1

#=======================================================
# PRE-PROCESSING

# checking number of instruments in a composition
instruments_list_without_drums = list(set([y[3] for y in events_matrix1 if y[3] != 9]))

instruments_list = list(set([y[3] for y in events_matrix1]))
num_instr = len(instruments_list)

# filtering out empty compositions and checking desired number of instruments in a composition
# It had been observed that music models learn best from multi-instrumental music, even for solo instruments
# So you can setup filtering by number of instruments here if you want

if len(events_matrix1) > 0 and len(instruments_list_without_drums) > 0:

    # recalculating timings
    for e in events_matrix1:
        e[1] = math.ceil(e[1] / 8) # Max 1 seconds for start-times
        e[2] = math.ceil(e[2] / 16) # Max 2 seconds for durations

    # Sorting by pitch, then by start-time
    events_matrix1.sort(key=lambda x: x[4], reverse=True)
    events_matrix1.sort(key=lambda x: x[1])

    # pitches augment stuff (calculating transpose value to C4 without drums)
    pitches = [y[4] for y in events_matrix1 if y[3] != 9]

    if len(pitches) > 0:
      avg_ptc = round(statistics.mean(pitches))
    else:
      avg_ptc = 0

    ptc_delta = 12 - (avg_ptc % 12)

    #=======================================================
    # FINAL PRE-PROCESSING

    first_event = True
    melody_chords = []
    pe = events_matrix1[0]
    pt = -1

    for e in events_matrix1:

        # Cliping all values...
        tim = max(0, min(127, e[1]-pe[1]))
        dur = max(1, min(127, e[2]))
        cha = max(0, min(11, e[3]))
        ptc = max(1, min(127, e[4]))
        vel = max(8, min(127, e[5]))

        velocity = round(vel / 15)

        # Pitches shifting
        if cha != 9:
            if transpose_to_model_average_pitch:
                ptc_aug = ptc + ptc_delta # Transposing composition to median C4
            else:
                ptc_aug = ptc
        else:
          ptc_aug = ptc + 128 # Shifting drums pitches because drums structure is different from non-drums

        # Time shifting
        if tim == 0 and first_event == True:
          abs_time = 0

        if tim == 0 and first_event == False:
          if pt == 0:
            pass
          else:
            abs_time += 128

        if tim !=0:
          abs_time = tim

        # Writing final note 
        melody_chords.append([abs_time, dur, cha, ptc_aug, velocity])

        pe = e
        pt = tim

        first_event = False

    #=======================================================
    # FINAL PROCESSING
    #=======================================================

    # Break between compositions / Intro seq

    # 758 == SOS/EOS token
    # 759 == SOS/EOS token
    # 760-761 == Composition is without drums or with drums
    # 762-773 == Number of instruments in a composition

    # TOTAL DICTIONARY SIZE OF 774 TOKENS

    if 9 in instruments_list:
      drums_present = 761 # Yes
    else:
      drums_present = 760 # No

    melody_chords_f.extend([758, 759, drums_present, 762+(num_instr-1)])
    melody_chords_f1.append([758, 759, drums_present, 762+(num_instr-1)])

    #=======================================================

    # Composition control seq
    intro_mode_time = statistics.mode([y[0] for y in melody_chords if y[2] != 9])
    intro_mode_dur = statistics.mode([y[1] for y in melody_chords if y[2] != 9])
    intro_mode_pitch = statistics.mode([y[3] for y in melody_chords if y[2] != 9])
    intro_mode_velocity = statistics.mode([y[4] for y in melody_chords if y[2] != 9])

    # Instrument value 12 is reserved for composition control seq
    intro_vel = (12 * 9) + intro_mode_velocity

    melody_chords_f.extend([intro_mode_time, intro_mode_dur+256, intro_mode_pitch+384, intro_vel+640])
    melody_chords_f1.append([intro_mode_time, intro_mode_dur+256, intro_mode_pitch+384, intro_vel+640])

    #=======================================================
    # MAIN PROCESSING CYCLE
    #=======================================================

    for m in melody_chords:

        # WRITING EACH NOTE HERE
        chan_vel = (m[2] * 9) + m[4]
        melody_chords_f.extend([m[0], m[1]+256, m[3]+384, chan_vel+640])
        melody_chords_f1.append([m[0], m[1]+256, m[3]+384, chan_vel+640])

    #=======================================================
print('Done!')

print('Notes:', len(melody_chords_f1), 'Tokens:', len(melody_chords_f))

# (GENERATE)

In [ ]:
#@title Standard/Simple Generation

def closest_acc_index(cba, acc):
    return min(range(len(cba)), key = lambda i: abs(cba[i]-acc))

model.eval()

num_prime_tokens = 512

num_batches = 1

num_blocks_to_generate = 1
block_size_in_tokens = 512

temperature = 0.8
min_stop_token = 758

outy = melody_chords_f[:num_prime_tokens]

inp = [outy] * num_batches


inp = torch.LongTensor(inp).cuda()

start_time = time()


out = model.generate(inp, 
                      block_size_in_tokens, 
                      temperature=temperature, 
                      return_prime=False, 
                      min_stop_token=min_stop_token, 
                      verbose=True)

inp = out.tolist()
outy.extend(inp[0])

print(time() - start_time, "seconds")

#======================================================================

out1 = outy

print('Sample INTs', out1[:15])

if len(out) != 0:
    
    song = out1
    song_f = []
    tim = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    son = []
    song1 = []

    for s in song:
      if s > 256 and s < (12*9)+640:
        son.append(s)
      else:
        if len(son) == 4:
          song1.append(son)
        son = []
        son.append(s)
                    

    for ss in song1:

        if ss[0] < 128:
          tim += ss[0] * 8

        dur = (ss[1]-256) * 16

        if (ss[2]-384) > 128:
          pitch = (ss[2]-384) - 128
        else:
          pitch = (ss[2]-384)


        channel = (ss[3]-640) // 9
        vel = ((ss[3]-640) % 9) * 15
                        
        song_f.append(['note', tim, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Orchestrator',  
                                                        output_file_name = '/content/Orchestrator-Music-Composition', 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)

    print('Done!')    

In [ ]:
#@title Stabilized Generation

def closest_acc_index(cba, acc):
    return min(range(len(cba)), key = lambda i: abs(cba[i]-acc))

model.eval()

num_prime_tokens = 512

num_batches = 3

num_blocks_to_generate = 128
block_size_in_tokens = 4

temperature = 0.7
min_stop_token = 0

outy = melody_chords_f[:num_prime_tokens]

inp = [outy] * num_batches
inp = torch.LongTensor(inp).cuda()
baseline_acc = model.choose_best_acc(inp)[0][2]

start_time = time()

stats = []

for i in tqdm.tqdm(range(num_blocks_to_generate)):

    out = model.generate(inp, 
                         block_size_in_tokens, 
                         temperature=temperature, 
                         return_prime=True, 
                         min_stop_token=min_stop_token, 
                         verbose=False)
    
    cba = model.choose_best_acc(out)
    acc_batch_number = closest_acc_index([y[2] for y in cba], baseline_acc)
    cba_idx = cba[acc_batch_number][0]
    stats.append(cba)
    
    inp = out[cba_idx].tolist()
    outy.extend(inp[-block_size_in_tokens:])
    inp = [inp] * num_batches
    
    inp = torch.LongTensor(inp).cuda()

print(time() - start_time, "seconds")
print([baseline_acc])
print(stats[0])
print(stats[-1])

#======================================================================

out1 = outy

print('Sample INTs', out1[:15])

if len(out) != 0:
    
    song = out1
    song_f = []
    tim = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    son = []
    song1 = []

    for s in song:
      if s > 256 and s < (12*9)+640:
        son.append(s)
      else:
        if len(son) == 4:
          song1.append(son)
        son = []
        son.append(s)
                    

    for ss in song1:

        if ss[0] < 128:
          tim += ss[0] * 8

        dur = (ss[1]-256) * 16

        if (ss[2]-384) > 128:
          pitch = (ss[2]-384) - 128
        else:
          pitch = (ss[2]-384)


        channel = (ss[3]-640) // 9
        vel = ((ss[3]-640) % 9) * 15
                        
        song_f.append(['note', tim, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Orchestrator',  
                                                        output_file_name = '/content/Orchestrator-Music-Composition', 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)

    print('Done!')    

In [ ]:
#@title Orchestrate

model.eval()

start_time = time()

inp = melody_chords_f[8:11] + [((8*9)+6)+640]
#inp = melody_chords_f[8:11] + [((8*9)+6)+640]

#inp = melody_chords_f[:11]

inp = torch.LongTensor(inp).cuda()

for i in tqdm.tqdm(range(3, len(melody_chords_f1[:256]))):

    out = model.generate(inp[None, ...], 
                         1, 
                         temperature=0.8, return_prime=False, min_stop_token=758, verbose=False)
    
    inp = torch.cat((inp, out[0]))
    
    next_note = torch.LongTensor(melody_chords_f1[i][:3]).cuda()
    
    inp = torch.cat((inp, next_note))

print(time() - start_time, "seconds")
print(model.choose_best_acc([inp[-SEQ_LEN:]]))

#======================================================================



out1 = inp.tolist()

print('Sample INTs', out1[:15])

if len(out) != 0:
    
    song = out1
    song_f = []
    tim = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    son = []
    song1 = []

    for s in song:
      if s > 256 and s < (12*9)+640:
        son.append(s)
      else:
        if len(son) == 4:
          song1.append(son)
        son = []
        son.append(s)
                    

    for ss in song1:

        if ss[0] < 128:
          tim += ss[0] * 8

        dur = (ss[1]-256) * 16

        if (ss[2]-384) > 128:
          pitch = (ss[2]-384) - 128
        else:
          pitch = (ss[2]-384)


        channel = (ss[3]-640) // 9
        vel = ((ss[3]-640) % 9) * 15
                        
        song_f.append(['note', tim, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Orchestrator',  
                                                        output_file_name = '/content/Orchestrator-Music-Composition', 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)

    print('Done!')

# Congrats! You did it! :)